 # Домашняя работа по теории вероятности


Задача: собрать цены квартир жилого комплекса *"Спутник"* вместе с дополнительной информацией и статически их проанализировать, найдя доверительный интервал цен двумя способами и сохранить сведения о квартирах, которые в них попали в их пересечение

## Часть 0
**Установка зависимостей**

In [12]:
#если нет pip в конде, то раскомментить строку ниже
#%conda install pip
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Часть 1
**Сбор данных**

In [11]:
from shutil import which
import pandas as pd
from bs4 import BeautifulSoup # pip install bs4
from selenium import webdriver # pip install selenium
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver_manager
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from time import sleep

In [2]:
# проверка, запускается ли код у меня или у другого с браузером вивальди
vivaldi_existance = which("vivaldi")
if vivaldi_existance:
    import chromedriver_py # pip install --user chromedriver-py==106.0.5249.61
    driverPATH = chromedriver_py.binary_path
    browserBIN = which("vivaldi")

In [3]:
# запуск драйвера и получение html кода
site = "https://samolet.ru/flats/?ordering=filter_price,pk&free=1&project=7"
# проверка, запускается ли код у меня или у другого с браузером вивальди
if vivaldi_existance:
    options = webdriver.ChromeOptions()
    options.binary_location = browserBIN
    driver = webdriver.Chrome(service=Service(driverPATH), options=options)
else: # если код запускается у человека с хромом
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(site) # запрос на переход на сайт
sleep(1.5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1800);") # проматывание до кнопки
button = driver.find_element(By.XPATH, '//button[@class="r-btn _size-m _primary"]') # создание интерактивного с кнопкой объекта
driver.execute_script("arguments[0].click();", button) # кликание по кнопке через объект
try:
    while button.is_displayed(): # пока кнопка будет отображаться, будем её кликать и матать вниз
        try:
            sleep(1.5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1800);") 
            button = driver.find_element(By.XPATH, '//button[@class="r-btn _size-m _primary"]') 
            sleep(0.5)
            driver.execute_script("arguments[0].click();", button) 
        except NoSuchElementException:
            break
except StaleElementReferenceException:
    pass
content = driver.page_source # извлечение html кода
driver.quit() # закрываем браузер
soup = BeautifulSoup(content, features="lxml") # вгоняем в суп
print('Информация получена!')

Информация получена!


## Часть 2
**Обработка данных**

На данном этапе мы пропарсим полученный html код и извлечем всю нужную информацию: цена, площадь, тип квартиры и ссылка на её покупку

In [70]:
# получаем список квартир
raw_info = soup.find('div', {'class':'flats-container__list-cards'})
list_of_flats = raw_info.find_all('div', {'class':'flats-container__list-card'})

In [71]:
links_to_purchase = []
prices = []
areas = []
types_of_flats = []
for flat in list_of_flats:
    try:
        links_to_purchase.append(flat.a['href'])
        card = flat.find('div', {'class':'card__wrap'})
        price_info = card.find('div', {'class':'card__price'})
        other_info = card.find('div', {'class':'card__container'}).ul.find_all('li')[0:2]
        price = int(price_info.span.text.replace('\n', '').strip().replace('\xa0₽', '').replace(' ', ''))
        type_of_flat = other_info[0].text.replace('\n', '').strip()
        area = float(other_info[1].text.replace('\n', '').strip().replace(' м²', '').replace(',','.'))
        prices.append(price)
        types_of_flats.append(type_of_flat)
        areas.append(area)
    except AttributeError:
        pass
    except TypeError:
        pass

In [77]:
flats_df = pd.DataFrame({
    "Тип квартиры": types_of_flats,
    "Цена": prices,
    "Площадь (м²)": areas,
    "Ссылка на покупку": links_to_purchase
})
flats_df

,Тип квартиры,Цена,Площадь (м²),Ссылка на покупку
0,Студия,5474332,19.20,https://samolet.ru/project/sputnik/flats/21092...
1,Студия,5661222,20.10,https://samolet.ru/project/sputnik/flats/21099...
2,Студия,5798398,20.20,https://samolet.ru/project/sputnik/flats/21105...
3,Студия,5851975,23.20,https://samolet.ru/project/sputnik/flats/20792...
4,Студия,6233690,24.60,https://samolet.ru/project/sputnik/flats/21079...
...,...,...,...,...
708,4-комн.,17488928,86.30,https://samolet.ru/project/sputnik/flats/21117...
709,4-комн.,17965247,84.57,https://samolet.ru/project/sputnik/flats/70471...
710,4-комн.,22662460,92.55,https://samolet.ru/project/sputnik/flats/12617...
711,4-комн.,22728540,92.55,https://samolet.ru/project/sputnik/flats/12619...


Получили выборку из 713 квартир. Приступим к статистической части

## Часть 3
**Получение доверительных интервалов**

Сначала найдем все статистические данные

In [89]:
prices = flats_df['Цена']
prices

0       5474332
1       5661222
2       5798398
3       5851975
4       6233690
         ...   
708    17488928
709    17965247
710    22662460
711    22728540
712    23789539
Name: Цена, Length: 713, dtype: int64

In [115]:
prices_mean = prices.mean()
print(f"Средняя цена - {prices_mean:.3f} рублей")
R = prices.max() - prices.min()
print(f"Размах - {R} рублей")
k_0 = R/prices_mean
print(f"Коэффициент относительного колебания значения крайних цен от средней - {k_0*100:.3f}%")
D_in = prices.var(ddof=0)
print(f"Неисправленная дисперсия - {D_in:.3f} рублей")
D = prices.var()
print(f"Исправленная дисперсия - {D:.3f} рублей")
prices_std = prices.std()
print(f"Стандартное отклонение - {prices_std:.3f} рублей")
var_q = prices_std/prices_mean
print(f"Коэффициент вариации - {var_q*100:.3f}%")

Средняя цена - 10343091.243 рублей
Размах - 18315207 рублей
Коэффициент относительного колебания значения крайних цен от средней - 177.077%
Неисправленная дисперсия - 6625117485369.463 рублей
Исправленная дисперсия - 6634422425657.903 рублей
Стандартное отклонение - 2575737.259 рублей
Коэффициент вариации - 24.903%


### Часть 3.1
*Способ первый - распределение Стьюдента*

In [98]:
import numpy as np
from scipy.stats import t

# зададим надежность 
confidence = 0.95
n_minus_one = len(prices)-1

# вычислим t
t_ = np.abs(t.ppf((1-confidence)/2, n_minus_one))

In [101]:
interval1_left, interval1_right = prices_mean-prices_std*t_/np.sqrt(len(prices)), prices_mean+prices_std*t_/np.sqrt(len(prices))
print(f"Полученный первым способом доверительный интервал: ({interval1_left:.3f}, {interval1_right:.3f})")

Полученный первым способом доверительный интервал: (10153707.030, 10532475.455)


In [111]:
# Данные, попавшие в первый доверительный интервал
first_interval_data = flats_df[(interval1_left<flats_df['Цена']) & (flats_df['Цена']<interval1_right)]
first_interval_data

,Тип квартиры,Цена,Площадь (м²),Ссылка на покупку
425,2-комн.,10179436,47.10,https://samolet.ru/project/sputnik/flats/20824...
426,2-комн.,10182764,46.60,https://samolet.ru/project/sputnik/flats/20833...
427,1-комн.,10184452,49.19,https://samolet.ru/project/sputnik/flats/11447...
428,2-комн.,10278038,47.30,https://samolet.ru/project/sputnik/flats/20830...
429,2-комн.,10289057,47.50,https://samolet.ru/project/sputnik/flats/20803...
430,2-комн.,10320072,49.85,https://samolet.ru/project/sputnik/flats/95107...
431,1-комн.,10332981,49.19,https://samolet.ru/project/sputnik/flats/11457...
432,2-комн.,10366584,54.30,https://samolet.ru/project/sputnik/flats/21098...
433,2-комн.,10366961,47.00,https://samolet.ru/project/sputnik/flats/20832...
434,2-комн.,10446219,50.70,https://samolet.ru/project/sputnik/flats/20809...


### Часть 3.2
*Способ первый - медиана и стандартное отклонение*

In [105]:
prices_median = prices.median()
interval2_left, interval2_right = prices_median-prices_std, prices_median+prices_std
print(f"Полученный первым способом доверительный интервал: ({interval2_left:.3f}, {interval2_right:.3f})")

Полученный первым способом доверительный интервал: (6701444.741, 11852919.259)


In [112]:
# Данные, попавшие во второй доверительный интервал
second_interval_data = flats_df[(interval2_left<flats_df['Цена']) & (flats_df['Цена']<interval2_right)]
second_interval_data

,Тип квартиры,Цена,Площадь (м²),Ссылка на покупку
17,Студия,6714467,24.60,https://samolet.ru/project/sputnik/flats/21085...
18,Студия,6761379,24.60,https://samolet.ru/project/sputnik/flats/21111...
19,Студия,6784028,25.60,https://samolet.ru/project/sputnik/flats/20792...
20,Студия,6838219,24.10,https://samolet.ru/project/sputnik/flats/20816...
21,Студия,6892499,25.10,https://samolet.ru/project/sputnik/flats/21089...
...,...,...,...,...
477,2-комн.,11725172,49.19,https://samolet.ru/project/sputnik/flats/11468...
478,2-комн.,11735854,48.95,https://samolet.ru/project/sputnik/flats/11474...
479,2-комн.,11792026,49.12,https://samolet.ru/project/sputnik/flats/11476...
480,2-комн.,11824839,49.12,https://samolet.ru/project/sputnik/flats/11479...


## Часть 4
**Выгрузка результатов**

Из полученных данных очевидно, что первый доверительный интервал полностью включен во второй. Значит, результатом будет являться первый доверительный интервал. Его сохраним в файле "main_result.csv". Но второй результат тоже сохраним в файле "additional_result.csv"

In [114]:
first_interval_data.to_csv("main_result.csv", index=False)
second_interval_data.to_csv("additional_result.csv", index=False)